In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

import time
import os
import glob
import rasterio

from natsort import natsorted
from shapely import geometry
from shapely.validation import make_valid

In [2]:
def extract_img_folder(folder) : 
    
    img_list = []
    
    for img in natsorted(glob.glob(folder+'/*.tif')):
        
        with rasterio.open(img) as f:
            img = f.read(1)
        
        img_list.append(img)

    return img_list

In [6]:
extract_img_folder(r'C:\Users\simon\OneDrive\2.ICE\MClureStrait_2024_05_05')

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)]

In [3]:
def read_img(namefile) : 
    
    with rasterio.open(namefile) as f:
            img = f.read(1)
            
    return img

In [8]:
read_img(r"C:\Users\simon\OneDrive\2.ICE\MClureStrait_2024_05_05\MClureStrait_2024_05_05.tif")

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [ ]:
def canny_edge_opencv(img_list, low, high, land_mask, fill_area = False) : 
    
    """
    Function that applies the canny edge algorithm to a list of radar images. 
    
    To the the value for the land to be, we use the value of two pixels taken in the ocean. 
    If the either one of the two is lower on the gray scale, the land will be blacker. 
    
    Inputs : 
        img_list -> list of images
        low -> lower threshold for the detection algorithm (int)
        high -> highest threshold for the detection algorithm (int)
        land_mask -> Binary mask differientiating the land from the ocean (1 : land, 0 : ocean)
        circle -> coordinates of the circle marking the radar rage (specified in the parameter file) (ints)
        fill_area -> Set to true when computing the complete field of view of the radar, otherwise set to false. 
        
    Returns:
        edges_list -> List of canny edge outputs corresponding to the img_list
        gray_list -> same as img_list but the images are now on the gray scale
    """

    
    #setting the initial parameters and list to be appended to
    it = 0
    edges_list = []
    gray_list = []
    
    #main loop
    for img in img_list : 
        
        print('Canny edge algorithm iteration : ', it)
        
        #Finding the land
        idx_land = np.where(land_mask == 1)
        
        #pixel test to determine the land fill value
        first_pixel_test  = np.mean(img[50, 250])
        second_pixel_test = np.mean(img[250, 50])
        
        if fill_area :
            fill_value = 255
        
        else : 

            if first_pixel_test < 35 or second_pixel_test < 35 : 
                fill_value = 15
            
            else : 
                fill_value = 56
            
        #setting the color of the land
        img[idx_land] = fill_value   

        #Computing the contours
        edges_img = cv.Canny(img, low, high)
            
        #appending to the lists
        edges_list.append(edges_img)
        gray_list.append(img)
        
        it += 1
        
    return edges_list, gray_list

In [9]:
img_list = extract_img_folder(r'C:\Users\simon\OneDrive\2.ICE\MClureStrait_2024_05_05')
land_mask = read_img(r"C:\Users\simon\OneDrive\2.ICE\MClureStrait_2024_05_05\MClureStrait_2024_05_05.tif")

canny_edge_opencv(img_list, 100, 200, land_mask, fill_area = False)

NameError: name 'canny_edge_opencv' is not defined